In [ ]:
%pip install python-dotenv langchain langchain-upstage langchain-community langchain-text-splitters 

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [2]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

# 환경변수를 불러옴
load_dotenv()

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = UpstageEmbeddings(model="solar-embedding-1-large")

In [7]:
from langchain_chroma import Chroma

# 데이터를 처음 저장할 때 
database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name='chroma-tax', persist_directory="./chroma")

# 이미 저장된 데이터를 사용할 때 
# database = Chroma(collection_name='chroma-tax', persist_directory="./chroma", embedding_function=embedding)

In [8]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
retrieved_docs = database.similarity_search(query, k=3)

In [ ]:
retrieved_docs

In [10]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage()

In [11]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [12]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [ ]:
ai_message = qa_chain({"query": query})

In [14]:
# 강의에서는 위처럼 진행하지만 업데이트된 LangChain 문법은 `.invoke()` 활용을 권장
ai_message = qa_chain.invoke({"query": query})

In [ ]:
ai_message